In [282]:
from pynput.mouse import Controller as Mouse, Button as MouseBtn
from pynput import keyboard
import time
import attrs

MOUSE = Mouse()
KEYBOARD = keyboard.Controller()

@attrs.frozen
class ActionSetup():
    dx: int
    dt: float

@attrs.frozen
class MouseSetup():
    # actions related
    slide: ActionSetup
    memory: ActionSetup
    drop: ActionSetup
    rotateDuration: float|None
    
    # env related
    deviceSize: tuple[int, int]
    isFullScreen: bool


TETRIS_FULLSCREEN = MouseSetup(
    slide=ActionSetup(65, 0.3),
    memory=ActionSetup(100, 0.1),
    drop=ActionSetup(100, 0.1),
    rotateDuration=0.01,
    deviceSize=(1000, 1000), isFullScreen=True)

CURRENT_SETUP = TETRIS_FULLSCREEN


In [283]:
def mouseDragVertical(
        dist:int, duration:float, goDown: bool, goBack: bool = True) -> None:
    dist = dist * (1 if goDown else -1)
    MOUSE.press(MouseBtn.left)
    MOUSE.move(0, dist)
    time.sleep(duration)
    MOUSE.release(MouseBtn.left)
    if goBack is True:
        MOUSE.move(0, -dist)


def decomposeMove(dPx: int, N: int) -> list[int]:
    assert N > 0
    q, r = divmod(abs(dPx), N)
    sign = (1 if dPx >= 0 else -1)
    steps = [sign * (q + 1) for _ in range(r)]
    steps.extend(sign * q for _ in range(N - r))
    return steps

def mouseDragSides(
        dx: int, duration: float, 
        nbSteps: int|None = None, goBack: bool = True) -> None:
    if nbSteps is None:
        # => move 1 px at each step
        nbSteps = abs(dx) 
    steps = decomposeMove(dx, nbSteps)
    MOUSE.press(MouseBtn.left)
    start = time.perf_counter()
    sleep_time = 0
    for i in range(nbSteps):
        MOUSE.move(steps[i], 0)
        target = start + (i + 1) * (duration / nbSteps)
        sleep_time = target - time.perf_counter()
        if sleep_time > 0:
            time.sleep(sleep_time)
    MOUSE.release(MouseBtn.left)
    if goBack is True:
        MOUSE.move(-dx, 0)


def slideBy(nbBlocks:int)->None:
    """slide the current block by `nbBlocks`
    `nbBlocks` < 0 => to the left | > 0 to the rigth | can't be 0"""
    assert nbBlocks != 0
    mouseDragSides(
        dx=nbBlocks*CURRENT_SETUP.slide.dx,
        duration=CURRENT_SETUP.slide.dt,
        nbSteps=None, goBack=True)

def dropBlock()->None:
    mouseDragVertical(
        dist=CURRENT_SETUP.drop.dx, 
        duration=CURRENT_SETUP.drop.dt,
        goDown=True, goBack=True)
    
def putInMemory()->None:
    mouseDragVertical(
        dist=CURRENT_SETUP.memory.dx, 
        duration=CURRENT_SETUP.memory.dt,
        goDown=False, goBack=True)

def rotateBy(nbRigthRotations:int)->None:
    for _ in range(nbRigthRotations):
        MOUSE.press(MouseBtn.left)
        if CURRENT_SETUP.rotateDuration is not None:
            time.sleep(CURRENT_SETUP.rotateDuration)
        MOUSE.release(MouseBtn.left)

In [284]:
from holo.protocols import _P, _T
from holo.__typing import Callable, Generic, Iterable
from holo.prettyFormats import prettyPrint, PrettyfyClass, prettyTime
from holo.profilers import SimpleProfiler, _ProfilerCategory, Profiler



class CallPack(Generic[_P, _T], PrettyfyClass):
    def __init__(self, func:Callable[_P, _T], *args:_P.args, 
                 **kwargs:_P.kwargs) -> None:
        self.func = func
        self.args = args
        self.kwargs = kwargs
    
    def call(self)->_T:
        return self.func(*self.args, **self.kwargs)

def waitPress(trueKey:str, falseKey:str) -> bool:
    result: bool | None = None
    def on_press(key: keyboard.KeyCode | keyboard.Key | None) -> bool:
        nonlocal result
        if not isinstance(key, (keyboard.KeyCode)):
            return True
        if key.char == trueKey:
            result = True
        elif key.char == falseKey:
            result = False
        else: 
            return True # continue
        return False  # Stop listener
    with keyboard.Listener(on_press=on_press) as listener: # type: ignore  
        listener.join()
    assert result is not None
    return result


def testCalls(configs:list[CallPack])->list[tuple[bool, CallPack]]:
    results: list[tuple[bool, CallPack]] = []
    for cfg in configs:
        cfg.call()
        success = waitPress(trueKey=",", falseKey=";")
        results.append((success, cfg))
    return results


def mesureExec(configs:list[CallPack], N:int):
    pt = prettyTime
    _prof = Profiler([])
    for cfg in configs:
        cat = _ProfilerCategory(_prof)
        for i in range(N):
            with SimpleProfiler() as sp:
                cfg.call()
            cat._update(sp.time())
        print(f"-> mean: {pt(cat.avgMesure())} +- {pt(cat.stdMesure())} for {cfg.kwargs}")


In [285]:
### tests  done with the 1000x1000 virtual device on my personal computer
def genOpposit(configs:list[CallPack])->list[CallPack]:
    res: list[CallPack] = []
    for cfg in configs:
        res.append(cfg)
        kwargs2 = cfg.kwargs.copy()
        kwargs2["dx"] *= -1
        res.append(CallPack(cfg.func, *cfg.args, **kwargs2))
    return res
def sign(dx:int):
    return (-1 if dx < 0 else +1)

#### try to move by 1 ####

# determine a good slow DX (found 60 to be working)
setup1 = genOpposit([
    CallPack(mouseDragSides, dx=dx, duration=0.3, nbSteps=None, goBack=True)
    for dx in [45, 50, 55, 60, 65, 70]])
# determine a good faster DX (found 60 to be working)
setup2 = genOpposit([
    CallPack(mouseDragSides, dx=dx, duration=0.1, nbSteps=None, goBack=True)
    for dx in [45, 50, 55, 60, 65, 70]])
# determine the nb of steps (found None is best)
setup3 = genOpposit([
    CallPack(mouseDragSides, dx=60, duration=0.3, nbSteps=i, goBack=True)
    for i in [1, 2, 3, 5, 10, 20, 30, 60, None]])
# now refine the dx (found 62 to be safe)
setup4 = genOpposit([
    CallPack(mouseDragSides, dx=dx, duration=0.1, nbSteps=None, goBack=True)
    for dx in [55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]])
# try to go even faster (0.05 is UNSTABLE, 0.1 is STABLE, deeper search maybe later)
setup5 = genOpposit([
    CallPack(mouseDragSides, dx=dx, duration=0.075, nbSteps=None, goBack=True)
    for dx in [55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]])


#### try to move by 3 (it is the max a piece will have to move) ####
def moveByXFunc(dx:int, duration:float, X:int):
    for _ in range(10):
        mouseDragSides(dx=65*sign(-dx), duration=0.2, nbSteps=None, goBack=True)
    mouseDragSides(dx=3*dx, duration=duration, nbSteps=None, goBack=True)

# determine a good DX
setup1_by3 = genOpposit([
    CallPack(moveByXFunc, dx=dx, duration=0.2, X=3)
    for dx in [55, 60, 65, 70, 75, 80]])


#### try to rotate a piece ####
def rotateTests(duration:float|None)->None:
    for _ in range(2):
        MOUSE.press(MouseBtn.left)
        if duration is not None: 
            time.sleep(duration)
        MOUSE.release(MouseBtn.left)

setup1_rotate = [CallPack(rotateTests, duration=dt) 
                 for dt in [0.1, 0.075, 0.05, 0.025, 0.01, 0.005, 0.001, None]]

In [237]:
prettyPrint(setup1, specificCompact={CallPack})

[
    CallPack(func=<function mouseDragSides at 0x0000014806E10900>, args=(), kwargs={dx: 45, duration: 0.3, nbSteps: None, goBack: True}),
    CallPack(func=<function mouseDragSides at 0x0000014806E10900>, args=(), kwargs={dx: -45, duration: 0.3, nbSteps: None, goBack: True}),
    CallPack(func=<function mouseDragSides at 0x0000014806E10900>, args=(), kwargs={dx: 50, duration: 0.3, nbSteps: None, goBack: True}),
    CallPack(func=<function mouseDragSides at 0x0000014806E10900>, args=(), kwargs={dx: -50, duration: 0.3, nbSteps: None, goBack: True}),
    CallPack(func=<function mouseDragSides at 0x0000014806E10900>, args=(), kwargs={dx: 55, duration: 0.3, nbSteps: None, goBack: True}),
    CallPack(func=<function mouseDragSides at 0x0000014806E10900>, args=(), kwargs={dx: -55, duration: 0.3, nbSteps: None, goBack: True}),
    CallPack(func=<function mouseDragSides at 0x0000014806E10900>, args=(), kwargs={dx: 60, duration: 0.3, nbSteps: None, goBack: True}),
    CallPack(func=<function m

In [239]:
time.sleep(5)
res = testCalls(setup1)
prettyPrint(res, specificCompact={tuple, dict}, specificFormats={CallPack: lambda x:x.kwargs})

[
    (False, {dx: 45, duration: 0.3, nbSteps: None, goBack: True}),
    (False, {dx: -45, duration: 0.3, nbSteps: None, goBack: True}),
    (False, {dx: 50, duration: 0.3, nbSteps: None, goBack: True}),
    (False, {dx: -50, duration: 0.3, nbSteps: None, goBack: True}),
    (True, {dx: 55, duration: 0.3, nbSteps: None, goBack: True}),
    (True, {dx: -55, duration: 0.3, nbSteps: None, goBack: True}),
    (True, {dx: 60, duration: 0.3, nbSteps: None, goBack: True}),
    (True, {dx: -60, duration: 0.3, nbSteps: None, goBack: True}),
    (True, {dx: 65, duration: 0.3, nbSteps: None, goBack: True}),
    (True, {dx: -65, duration: 0.3, nbSteps: None, goBack: True}),
    (True, {dx: 70, duration: 0.3, nbSteps: None, goBack: True}),
    (True, {dx: -70, duration: 0.3, nbSteps: None, goBack: True})
]


In [ ]:
time.sleep(5)
mesureExec(setup1, N=3)

-> mean: 301.453 ms +- 0.716 ms for {'dx': 45, 'duration': 0.3, 'nbSteps': None, 'goBack': True}
-> mean: 301.027 ms +- 80.036 μs for {'dx': -45, 'duration': 0.3, 'nbSteps': None, 'goBack': True}
-> mean: 301.098 ms +- 0.102 ms for {'dx': 50, 'duration': 0.3, 'nbSteps': None, 'goBack': True}
-> mean: 300.941 ms +- 0.142 ms for {'dx': -50, 'duration': 0.3, 'nbSteps': None, 'goBack': True}
-> mean: 300.972 ms +- 0.298 ms for {'dx': 55, 'duration': 0.3, 'nbSteps': None, 'goBack': True}
-> mean: 301.152 ms +- 0.190 ms for {'dx': -55, 'duration': 0.3, 'nbSteps': None, 'goBack': True}
-> mean: 301.010 ms +- 0.276 ms for {'dx': 60, 'duration': 0.3, 'nbSteps': None, 'goBack': True}
-> mean: 301.500 ms +- 1.028 ms for {'dx': -60, 'duration': 0.3, 'nbSteps': None, 'goBack': True}
-> mean: 301.174 ms +- 0.449 ms for {'dx': 65, 'duration': 0.3, 'nbSteps': None, 'goBack': True}
-> mean: 301.218 ms +- 0.443 ms for {'dx': -65, 'duration': 0.3, 'nbSteps': None, 'goBack': True}
-> mean: 301.132 ms +- 0

In [ ]:
time.sleep(5)
res = testCalls(setup2)
prettyPrint(res, specificCompact={tuple, dict}, specificFormats={CallPack: lambda x:x.kwargs})

[
    (False, {dx: 45, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: -45, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: 50, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: -50, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: 55, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: -55, duration: 0.1, nbSteps: None, goBack: True}),
    (True, {dx: 60, duration: 0.1, nbSteps: None, goBack: True}),
    (True, {dx: -60, duration: 0.1, nbSteps: None, goBack: True}),
    (True, {dx: 65, duration: 0.1, nbSteps: None, goBack: True}),
    (True, {dx: -65, duration: 0.1, nbSteps: None, goBack: True}),
    (True, {dx: 70, duration: 0.1, nbSteps: None, goBack: True}),
    (True, {dx: -70, duration: 0.1, nbSteps: None, goBack: True})
]


In [ ]:
time.sleep(5)
mesureExec(setup2, N=3)

-> mean: 101.257 ms +- 0.346 ms for {'dx': 45, 'duration': 0.1, 'nbSteps': None, 'goBack': True}
-> mean: 101.121 ms +- 0.185 ms for {'dx': -45, 'duration': 0.1, 'nbSteps': None, 'goBack': True}
-> mean: 101.090 ms +- 0.308 ms for {'dx': 50, 'duration': 0.1, 'nbSteps': None, 'goBack': True}
-> mean: 101.101 ms +- 0.355 ms for {'dx': -50, 'duration': 0.1, 'nbSteps': None, 'goBack': True}
-> mean: 101.247 ms +- 0.293 ms for {'dx': 55, 'duration': 0.1, 'nbSteps': None, 'goBack': True}
-> mean: 101.076 ms +- 0.207 ms for {'dx': -55, 'duration': 0.1, 'nbSteps': None, 'goBack': True}
-> mean: 101.517 ms +- 0.958 ms for {'dx': 60, 'duration': 0.1, 'nbSteps': None, 'goBack': True}
-> mean: 101.179 ms +- 0.293 ms for {'dx': -60, 'duration': 0.1, 'nbSteps': None, 'goBack': True}
-> mean: 101.226 ms +- 0.477 ms for {'dx': 65, 'duration': 0.1, 'nbSteps': None, 'goBack': True}
-> mean: 101.191 ms +- 0.322 ms for {'dx': -65, 'duration': 0.1, 'nbSteps': None, 'goBack': True}
-> mean: 101.136 ms +- 0.

In [ ]:
time.sleep(5)
res = testCalls(setup3)
prettyPrint(res, specificCompact={tuple, dict}, specificFormats={CallPack: lambda x:x.kwargs})

[
    (False, {dx: 60, duration: 0.3, nbSteps: 1, goBack: True}),
    (False, {dx: -60, duration: 0.3, nbSteps: 1, goBack: True}),
    (False, {dx: 60, duration: 0.3, nbSteps: 2, goBack: True}),
    (False, {dx: -60, duration: 0.3, nbSteps: 2, goBack: True}),
    (False, {dx: 60, duration: 0.3, nbSteps: 3, goBack: True}),
    (False, {dx: -60, duration: 0.3, nbSteps: 3, goBack: True}),
    (False, {dx: 60, duration: 0.3, nbSteps: 5, goBack: True}),
    (False, {dx: -60, duration: 0.3, nbSteps: 5, goBack: True}),
    (True, {dx: 60, duration: 0.3, nbSteps: 10, goBack: True}),
    (True, {dx: -60, duration: 0.3, nbSteps: 10, goBack: True}),
    (True, {dx: 60, duration: 0.3, nbSteps: 20, goBack: True}),
    (True, {dx: -60, duration: 0.3, nbSteps: 20, goBack: True}),
    (True, {dx: 60, duration: 0.3, nbSteps: 30, goBack: True}),
    (True, {dx: -60, duration: 0.3, nbSteps: 30, goBack: True}),
    (True, {dx: 60, duration: 0.3, nbSteps: 60, goBack: True}),
    (True, {dx: -60, duration: 

In [ ]:
time.sleep(5)
mesureExec(setup3, N=3)

-> mean: 303.034 ms +- 3.163 ms for {'dx': 60, 'duration': 0.3, 'nbSteps': 1, 'goBack': True}
-> mean: 300.953 ms +- 0.275 ms for {'dx': -60, 'duration': 0.3, 'nbSteps': 1, 'goBack': True}
-> mean: 301.077 ms +- 0.132 ms for {'dx': 60, 'duration': 0.3, 'nbSteps': 2, 'goBack': True}
-> mean: 301.049 ms +- 0.241 ms for {'dx': -60, 'duration': 0.3, 'nbSteps': 2, 'goBack': True}
-> mean: 301.105 ms +- 0.404 ms for {'dx': 60, 'duration': 0.3, 'nbSteps': 3, 'goBack': True}
-> mean: 301.148 ms +- 0.150 ms for {'dx': -60, 'duration': 0.3, 'nbSteps': 3, 'goBack': True}
-> mean: 300.930 ms +- 88.001 μs for {'dx': 60, 'duration': 0.3, 'nbSteps': 5, 'goBack': True}
-> mean: 302.198 ms +- 2.205 ms for {'dx': -60, 'duration': 0.3, 'nbSteps': 5, 'goBack': True}
-> mean: 300.888 ms +- 0.170 ms for {'dx': 60, 'duration': 0.3, 'nbSteps': 10, 'goBack': True}
-> mean: 301.104 ms +- 0.184 ms for {'dx': -60, 'duration': 0.3, 'nbSteps': 10, 'goBack': True}
-> mean: 300.741 ms +- 0.183 ms for {'dx': 60, 'dura

In [ ]:
time.sleep(5)
res = testCalls(setup4)
prettyPrint(res, specificCompact={tuple, dict}, specificFormats={CallPack: lambda x:x.kwargs})

[
    (False, {dx: 55, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: -55, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: 56, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: -56, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: 57, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: -57, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: 58, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: -58, duration: 0.1, nbSteps: None, goBack: True}),
    (True, {dx: 59, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: -59, duration: 0.1, nbSteps: None, goBack: True}),
    (True, {dx: 60, duration: 0.1, nbSteps: None, goBack: True}),
    (True, {dx: -60, duration: 0.1, nbSteps: None, goBack: True}),
    (False, {dx: 61, duration: 0.1, nbSteps: None, goBack: True}),
    (True, {dx: -61, duration: 0.1, nbSteps: None, goBack: True}),
    (True, {dx: 62, duration: 0.1, nbSteps: None, goBack:

In [211]:
time.sleep(5)
res = testCalls(setup5)
prettyPrint(res, specificCompact={tuple, dict}, specificFormats={CallPack: lambda x:x.kwargs})

[
    (False, {dx: 55, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: -55, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: 56, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: -56, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: 57, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: -57, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: 58, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: -58, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: 59, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: -59, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: 60, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: -60, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: 61, duration: 0.075, nbSteps: None, goBack: True}),
    (False, {dx: -61, duration: 0.075, nbSteps: None, goBack: True}),
    (True, {dx: 62, durat

In [226]:
time.sleep(5)
res = testCalls(setup1_by3)
prettyPrint(res, specificCompact={tuple, dict}, specificFormats={CallPack: lambda x:x.kwargs})

[
    (False, {dx: 55, duration: 0.2, X: 3}),
    (False, {dx: -55, duration: 0.2, X: 3}),
    (True, {dx: 60, duration: 0.2, X: 3}),
    (True, {dx: -60, duration: 0.2, X: 3}),
    (True, {dx: 65, duration: 0.2, X: 3}),
    (True, {dx: -65, duration: 0.2, X: 3}),
    (True, {dx: 70, duration: 0.2, X: 3}),
    (True, {dx: -70, duration: 0.2, X: 3}),
    (False, {dx: 75, duration: 0.2, X: 3}),
    (False, {dx: -75, duration: 0.2, X: 3}),
    (True, {dx: 80, duration: 0.2, X: 3}),
    (True, {dx: -80, duration: 0.2, X: 3})
]


In [246]:
time.sleep(5)
res = testCalls(setup1_rotate)
prettyPrint(res, specificCompact={tuple, dict}, specificFormats={CallPack: lambda x:x.kwargs})

[
    (True, {duration: 0.1}),
    (True, {duration: 0.075}),
    (True, {duration: 0.05}),
    (True, {duration: 0.025}),
    (False, {duration: 0.01}),
    (True, {duration: 0.005}),
    (True, {duration: 0.001}),
    (True, {duration: None})
]


In [247]:
time.sleep(5)
mesureExec(setup1_rotate, N=3)

-> mean: 203.632 ms +- 3.590 ms for {'duration': 0.1}
-> mean: 151.920 ms +- 0.360 ms for {'duration': 0.075}
-> mean: 101.401 ms +- 0.363 ms for {'duration': 0.05}
-> mean: 51.487 ms +- 0.275 ms for {'duration': 0.025}
-> mean: 21.417 ms +- 0.386 ms for {'duration': 0.01}
-> mean: 11.310 ms +- 0.310 ms for {'duration': 0.005}
-> mean: 3.443 ms +- 0.658 ms for {'duration': 0.001}
-> mean: 0.932 ms +- 0.167 ms for {'duration': None}


In [212]:
# researche for better sleep control
def action() -> None:
    # Simulate a fast operation
    pass

def run_precise_loop(duration: float, N: int) -> None:
    start = time.perf_counter()
    step = duration / N
    nbNoSleep = 0
    targets = []
    real = []
    

    for i in range(N):
        action()
        target = start + (i+1) * step
        sleep_time = target - time.perf_counter()
        if sleep_time > 0:
            t0 = time.perf_counter()
            time.sleep(sleep_time)
            real.append(time.perf_counter() - t0)
            targets.append(sleep_time)
        else: nbNoSleep += 1

    end = time.perf_counter()
    mean = lambda x: prettyTime(sum(x)/len(x))
    oversleep = prettyTime((sum(real)/len(real)) - (sum(targets)/len(targets)))
    realDuration = (end - start)
    print(f"Requested duration: {prettyTime(duration)}")
    print(f"Actual duration:   {prettyTime(realDuration)} with {nbNoSleep=}")
    print(f"error: {prettyTime(duration - realDuration)}")
    print(f"total slept time: {prettyTime(sum(real))}")
    print(f"average time slept: {mean(real)=!s} | {mean(targets)=!s} | {oversleep=!s}")

run_precise_loop(0.3, 28)

Requested duration: 300.000 ms
Actual duration:   300.263 ms with nbNoSleep=0
error: -2.633e-04 sec
total slept time: 300.194 ms
average time slept: mean(real)=10.721 ms | mean(targets)=10.436 ms | oversleep=0.285 ms


In [ ]:
# test multiple actions:
time.sleep(5)

putInMemory()
time.sleep(0.25)

rotateBy(3)
time.sleep(0.1)

slideBy(-3)
time.sleep(0.05)

dropBlock()